# ETL Pipeline

This notebook is part of my portfolio to demonstrate my skills in _data analytics_ and _data science_. Here I can demonstrate all the knowledge acquired in my career as a mechatronics engineer and the various courses in the work area such as Python SQL and Power BI.

They are used:

- SQL server 2022
- SQL Server Management Studio (SSMS) 20.2
- Power BI 2.138
- Python 3.12.7

In [1]:
# import the libraries to be use
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import psycopg2

## Extracting data from various source

### Extracting data from csv file using Pandas

In [ ]:
#Using library os, get the information from the files in the folder and print the folder and files names 
import os
for dirfolder, _, filenames in os.walk('/data'):
    for name in filenames:
        print(os.path.join(dirfolder, name))

In [ ]:
#Open the file CSV using pandas and save the data in a dataframe
dataframe = pd.read_csv('../dataset.csv')

In [ ]:
# View the dataframe column headers
dataframe.head()
# The structure of the dataframe
dataframe.shape

In [ ]:
# count the number of null values in the data set
dataframe.isnull().sum()

## Connection to Database in SQL

In [ ]:
try:
    global connection
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',
        password = '1234',
        database = database
    )
    print("Correct connection to database")
    #return True

except Exception as ex:
    print(f" Error connecting to database: {ex}")
    #return False